In [ ]:
import os
import re

In [ ]:
with open("./index_byc.csv", "r") as fp:
    lines = fp.readlines()
fp.close()

In [ ]:
print('all time sermon count:',
    len( lines )
     )

In [ ]:
def cleanse_p_tag_span_tag(inputText):
    txt2 = re.sub(r'<.+?>', '', inputText)
    return txt2

In [ ]:
def cleanse_nbsp_quot(inputText):
    txt2 = inputText \
        .replace('&nbsp;', '') \
        .replace('&quot;', '\'') \
        .replace('&#39;', '\'') \
        .replace('&ldquo;', '``') \
        .replace('&rdquo;', '"') \
        .replace('&lsquo;', '`') \
        .replace('&rsquo;', '\'') \
        .replace('&hellip;', '...') \
        .replace('&Omicron;', '零') \
        .replace('&mdash;', '─') \
        .replace('&ndash;', '─') \
        .replace('\\tab', ' ') \
        .replace('\\cs16', '') \
        .replace(' &divide;', '$\\div$') \
        .replace(' X ', '$\\times$') \
        .replace('&eacute;', '\\\'e') \
        .replace('ἵ', 'ι') \
        .replace('&nu;', 'ν') \
        .replace('&alpha;', 'α') \
        .replace('&beta;', 'β') \
        .replace('&gamma;', 'γ') \
        .replace('&delta;', 'δ') \
        .replace('&epsilon;', 'ε') \
        .replace('&sigma;', 'σ') \
        .replace('&iota;', 'ι') \
        .replace('&sigmaf;', 'ς') \
        .replace('-&gt', '$\rightarrow$') \
        .replace('שְׁאוֹל&lrm;', '\sblgoodhebrew{שְׁאוֹל}') \
        .replace('&middot;', '$\\,\\cdot\\,$') \
        .strip()
    return txt2

In [ ]:
def sermonContentRetrieval(pathfilename):
    with open(pathfilename, "r") as fp:
        lines = fp.readlines()
    fp.close()
    sermon_text = []
    for line in lines:
        if "<p style=" in line and "text-align:" in line and "justify;" in line:
            # print((cleanse_p_tag_span_tag(line)).strip())
            line = (cleanse_p_tag_span_tag(line)).strip()
            line = cleanse_nbsp_quot(line).strip()
            sermon_text.append(line)
    return sermon_text

In [ ]:
progressStepCnt = 0
# --------------------------------------
# read the index table and only take
# --------------------------------------
progressStepCnt += 1
print(f"Step {progressStepCnt}: reading in full index file")
with open("./index_byc.csv", "r") as fp:
    lines = fp.readlines()
fp.close()

In [ ]:
sermon_tex_filepath = '../../build/HKBC/hkbc_sermon.tex'
# --------------------------------------
# print the latex document : prefix
# --------------------------------------
progressStepCnt += 1
print(f"Step {progressStepCnt}: printing out prefixing")
_ = os.system(
    "cat ../prefix.tex | sed 's/講道逐字稿/港九培靈研經會 講道/' > " \
    + sermon_tex_filepath
)

progressStepCnt += 1
print(f"Step {progressStepCnt}: generate main content")
hkbc_path = '../../data/HKBC/'
s_prev = '' # previous HKBC html conference session & lecture number id code
s_curr = '' # current HKBC html conference session & lecture number id code
s_next = '' # next HKBC html conference session & lecture number id code

# --------------------------------------
# lines is the line content in index_byc
# --------------------------------------
for lineId in range(len(lines)):
    if (lineId+1) % 100 == 0:
        print(f"{lineId+1} of {len(lines)}")
    line = lines[lineId]
    line_contents = line.split(",")
    s_curr = line_contents[0]
    s_prev = lines[(lineId-1)%len(lines)].split(",")[0]
    s_next = lines[(lineId+1)%len(lines)].split(",")[0]
    if os.path.isfile(f'../../data/HKBC/{s_curr}'):
        # --------------------------------------
        # print out lecture title and misc. info
        # --------------------------------------
        with open(sermon_tex_filepath, "a") as fp:
            sectionNameStr = ''
            p_curr = line_contents[1] # preacher name
            cn_curr = line_contents[2] # conference no.
            ln_curr = line_contents[3] # lecture no.
            t_curr = line_contents[4] # full title
            sectionNameStr = f'第{cn_curr}屆 港九培靈會 {p_curr} {t_curr.split(" ", 1)[0]}'
            fp.write("\n\n\\section{"+sectionNameStr+"}\n")
            fp.write("\\label{sec:"+s_curr+"}\n")
            fp.write("\\textbf{"+t_curr.split(" ", 1)[-1]+"}\n")
            fp.write("\\newline\n\\newline\n")
            fp.write(
                "連結: \\href{https://www.hkbibleconference.org/session-message/view/" \
                +s_curr \
                +"}{\\texttt{https://www.hkbibleconference.org/session-message/view/" \
                +s_curr \
                +"}} \n"
            )
            fp.write("\\newline\n\\newline\n")
            fp.write("\\hyperref[sec:"+s_prev+"]{< < < PREV SERMON < < <}\n")
            fp.write("~\n")
            fp.write("\\hyperref[sec:"+s_next+"]{> > > NEXT SERMON > > >}\n")
            fp.write("\\newline\n\\newline\n")
        fp.close()
        # --------------------------------------
        # retrieve lecture content
        # --------------------------------------
        pathfilename = f"{hkbc_path}{s_curr}"
        sermon_text_lines = sermonContentRetrieval(pathfilename)
        with open(sermon_tex_filepath, "a") as fp:
            for sermon_text_line in sermon_text_lines:
                text_curr = sermon_text_line.strip()
                if len(text_curr) == 0:
                    continue
                fp.write(text_curr)
                fp.write("\n\\newline\n\\newline\n")
            fp.close()
progressStepCnt += 1
print(f"Step {progressStepCnt}: generate afterward and postfix")
# --------------------------------------
# print the latex document : afterword
# --------------------------------------
_ = os.system("cat ../afterword.tex >> " + sermon_tex_filepath)
# --------------------------------------
# print the latex document : postfix
# --------------------------------------
_ = os.system("cat ../postfix.tex >> " + sermon_tex_filepath)
print("done !")